# Retrieve Demo over Wikipedia

This example demonstrates how to setup mindsearch and allows to retrieve simple wikipedia.

Users can input a query or a question, this then uses semantic search to find relevant passages.

In [6]:
import os
import json
import gzip
import numpy as np

import mindspore
from mindspore import Tensor
from mindsearch.inference.inference import BaseBiEncoderInference
from mindsearch.bi_encoder.faiss_retriever import BaseFaissIPRetriever, search_queries
from mindsearch.utils.data_utils import download_url

First, download the pretrained model.

In [7]:
model_ckpt_url = "https://download.mindspore.cn/toolkits/mindsearch/retromae_base/mindspore_model.ckpt"
save_model_path = "retromae_base.ckpt"

if not os.path.exists(save_model_path):
    download_url(model_ckpt_url, save_model_path)

Then users should provide documents as the retrieve corpus. In this demo, we take wikipedia as an example.

In [12]:
vocab_path = "vocab.txt"
config_path="config.json"
wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'

if not os.path.exists(wikipedia_filepath):
    download_url('http://sbert.net/datasets/simplewiki-2020-11-01.jsonl.gz', wikipedia_filepath)

passages = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf-8') as reader:
    for line in reader:
        data = json.loads(line.strip())
        passages.append(data['paragraphs'][0])

print("Passages: ", len(passages))

Passages:  169597


In [13]:
mindspore.set_context(mode=mindspore.PYNATIVE_MODE, device_target="CPU")

All the documents will be inferenced by the model, and each document corresponds to a vector. Then the index can be built.

In [14]:
inference = BaseBiEncoderInference(save_model_path, config_file=config_path, vocab_file=vocab_path)
p_reps = inference.passage_batch_encode(passages, batch_size=8, use_tokenizer=True)

p_reps = np.array(p_reps).astype('float32')
retriever = BaseFaissIPRetriever(p_reps)

2022-11-03 20:51:54,314 - modeling.py[line:175] - INFO : try loading tied weight
2022-11-03 20:51:54,315 - modeling.py[line:176] - INFO : loading model weight from retromae_base.ckpt
[WARNING] ME(33568:33280,MainProcess):2022-11-03-20:52:00.629.472 [mindspore\dataset\engine\datasets_user_defined.py:656] Python multiprocessing is not supported on Windows platform.


Finally, search by the input query, return the search results. 

In [15]:
query = "Paris eiffel tower"
q_reps = inference.query_single_encode(query)

all_scores, psg_indices = search_queries(retriever, Tensor.asnumpy(q_reps))

for indice in psg_indices.tolist()[0][:20]:
    print(passages[indice])

Michael Te-Pei Chang (; born February 22, 1972) is an American retired tennis player. He was the youngest-ever male winner of a Grand Slam singles title. Michael Chang did this when he won the French Open in 1989 at the age of 17.
Equus is a genus of mammals in the family Equidae. It includes horses, asses, and zebras. "Equus" is the only living (extant) genus of horses, and there are seven living species. They are the one-toed horses, and are adapted for living in various types of grasslands.
Ted Cassidy (July 31, 1932 - January 16, 1979) was an American actor. He was best known for his roles as Lurch and Thing on "The Addams Family".
Catalão is a Brazilian city in the state of Goiás. It has 75.623 inhabitants. It covers . It was founded in 1833 and is today an important industrial center of state of Goiás.
216 Kleopatra is a Main belt asteroid found by Johann Palisa on April 10, 1880 in Pola. It is named after Cleopatra, the Queen of Egypt.
In music, a suite (pronounce "sweet") is a 

You can use func`cosine_similarity` to calculate vector similarity

In [16]:
from mindsearch.utils.model_utils import cosine_similarity
answer = "Paris eiffel tower"
a_reps = inference.query_single_encode(answer)
print("similar score: ", cosine_similarity(q_reps, a_reps))

similar score:  [[0.99999976]]
